<a href="https://colab.research.google.com/github/FlowSight/MlAlgoFromScratch/blob/master/Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import csv
from sklearn.metrics import log_loss,roc_auc_score
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error
from collections import OrderedDict
from scipy import linalg
import numpy as np
from itertools import combinations_with_replacement

In [ ]:
!wget https://raw.githubusercontent.com/FlowSight/dataset/master/data_banknote_authentication.txt
!wget https://raw.githubusercontent.com/eriklindernoren/ML-From-Scratch/master/mlfromscratch/data/TempLinkoping2016.txt

--2020-04-26 21:04:15--  https://raw.githubusercontent.com/FlowSight/dataset/master/data_banknote_authentication.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46400 (45K) [text/plain]
Saving to: ‘data_banknote_authentication.txt’

data_banknote_authe 100%[===================>]  45.31K  --.-KB/s    in 0.02s   

2020-04-26 21:04:16 (2.17 MB/s) - ‘data_banknote_authentication.txt’ saved [46400/46400]

--2020-04-26 21:04:17--  https://raw.githubusercontent.com/eriklindernoren/ML-From-Scratch/master/mlfromscratch/data/TempLinkoping2016.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP r

In [ ]:
class Sigmoid():
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))

    def gradient(self, x):
        return self.__call__(x) * (1 - self.__call__(x))

class Softmax():
    def __call__(self, x):
        e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return e_x / np.sum(e_x, axis=-1, keepdims=True)

    def gradient(self, x):
        p = self.__call__(x)
        return p * (1 - p)

class Loss(object):
    def loss(self, y_true, y_pred):
        return NotImplementedError()

    def gradient(self, y, y_pred):
        raise NotImplementedError()

    def acc(self, y, y_pred):
        return 0

class SquareLoss(Loss):
    def __init__(self): pass

    def loss(self, y, y_pred):
        return 0.5 * np.power((y - y_pred), 2)

    def gradient(self, y, y_pred):
        return -(y - y_pred)

    def hessian(self, y, y_pred):
        return 1


class CrossEntropy(Loss):
    def __init__(self): pass

    def loss(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - y * np.log(p) - (1 - y) * np.log(1 - p)

    def acc(self, y, p):
        return accuracy_score(np.argmax(y, axis=1), np.argmax(p, axis=1))

    def gradient(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - (y / p) + (1 - y) / (1 - p)

    def hessian(self,y,p):
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return y/(p**2) + (1-y)/(1-p)**2

class LogisticLoss():
    def __init__(self):
        sigmoid = Sigmoid()
        self.log_func = sigmoid
        self.log_grad = sigmoid.gradient

    def loss(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        p = self.log_func(y_pred)
        return y * np.log(p) + (1 - y) * np.log(1 - p)

    # gradient w.r.t y_pred
    def gradient(self, y, y_pred):
        p = self.log_func(y_pred)
        return -(y - p)

    # w.r.t y_pred
    def hessian(self, y, y_pred):
        p = self.log_func(y_pred)
        return p * (1 - p)


Gradient Boosting forest implementation

In [ ]:

import numpy as np
import math
import progressbar

def split_on_feature(X, feature_i, feature_threhold):
    split_func = None
    if isinstance(feature_threhold, int) or isinstance(feature_threhold, float):
        split_func = lambda sample: sample[feature_i] <= feature_threhold
    else:
        split_func = lambda sample: sample[feature_i] == feature_threhold

    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])

    return np.array([X_1, X_2])

def calculate_entropy(y):
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    entropy = 0
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        entropy += -p * log2(p)
    return entropy

def calculate_variance(X):
    mean = np.ones(np.shape(X)) * X.mean(0)
    n_samples = np.shape(X)[0]
    variance = (1 / n_samples) * np.diag((X - mean).T.dot(X - mean))
    return variance

def to_categorical(x, n_col=None):
    if not n_col:
        n_col = np.amax(x) + 1
    one_hot = np.zeros((x.shape[0], n_col))
    one_hot[np.arange(x.shape[0]), x] = 1
    return one_hot

class DecisionNode():
    def __init__(self, feature_i=None, threshold=None,
                 value=None, left=None, right=None, weight = None):
        self.feature_i = feature_i          # Index for the feature that is tested
        self.threshold = threshold          # Threshold value for feature
        self.value = value                  # Value if the node is a leaf in the tree
        self.left = left   
        self.right = right
        #self.weight - weight  # to be used for gbm/xgboost


class DecisionTree(object):

   def __init__(self,classify=False, min_samples_split=3, min_impurity=1e-5,
                 max_depth=float("inf"), loss=None, tree_lambda=1e-2):
        self.root = None 
        self.classify=classify
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        self.max_depth = max_depth
        if self.classify:
          self.impurity_calculation = self.information_gain
          self.leaf_value_calculation = self.majority_vote
        else:
          self.impurity_calculation = self.variance_reduction
          self.leaf_value_calculation = self.mean_of_y
        self.one_dim = None
        self.loss = loss
        self.tree_lambda = tree_lambda


   def information_gain(self, y, y1, y2):
        p = len(y1) / len(y)
        entropy = calculate_entropy(y)
        info_gain = entropy -p*calculate_entropy(y1) -(1-p)*calculate_entropy(y2)
        return info_gain
      
   def variance_reduction(self, y, y1, y2):
        var_tot = calculate_variance(y)
        var_1 = calculate_variance(y1)
        var_2 = calculate_variance(y2)
        frac_1 = len(y1) / len(y)
        frac_2 = len(y2) / len(y)
        variance_reduction = var_tot - (frac_1 * var_1 + frac_2 * var_2)
        return sum(variance_reduction)

   def majority_vote(self, y):
        most_common = None
        max_count = 0
        for label in np.unique(y):
            count = len(y[y == label])
            if count > max_count:
                most_common = label
                max_count = count
        return most_common

   def mean_of_y(self, y):
        value = np.mean(y, axis=0)
        return value if len(value) > 1 else value[0]

   def fit(self, X, y, loss=None):
        self.one_dim = len(np.shape(y)) == 1
        self.root = self.build_tree(X, y)
        self.loss=None

   def build_tree(self, X, y, current_depth=0):
        largest_impurity = 0
        best_criteria = None    # Feature index and threshold
        best_sets = None        # Subsets of the data

        # sanitize y
        if len(np.shape(y)) == 1:
            y = np.expand_dims(y, axis=1)
        np.reshape(y,(y.shape[0],y.shape[1]))
      
        Xy = np.concatenate((X, y), axis=1)
        n_samples, n_features = np.shape(X)

        if n_samples >= self.min_samples_split and current_depth <= self.max_depth:
            for feature_i in range(n_features):
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)
                for threshold in unique_values:
                    Xy1, Xy2 = split_on_feature(Xy, feature_i, threshold)

                    if len(Xy1) > 0 and len(Xy2) > 0:
                        # Select the y-values of the two sets
                        y1 = Xy1[:, n_features:]
                        y2 = Xy2[:, n_features:]

                        # Calculate impurity
                        impurity = self.impurity_calculation(y, y1, y2)
                        if impurity > largest_impurity:
                            largest_impurity = impurity
                            best_criteria = {"feature_i": feature_i, "threshold": threshold}
                            best_sets = {
                                "leftX": Xy1[:, :n_features],   # X of left subtree
                                "lefty": Xy1[:, n_features:],   # y of left subtree
                                "rightX": Xy2[:, :n_features],  # X of right subtree
                                "righty": Xy2[:, n_features:]   # y of right subtree
                                }

        if largest_impurity > self.min_impurity:
            left_branch = self.build_tree(best_sets["leftX"], best_sets["lefty"], current_depth + 1)
            right_branch = self.build_tree(best_sets["rightX"], best_sets["righty"], current_depth + 1)
            return DecisionNode(feature_i=best_criteria["feature_i"], threshold=best_criteria[
                                "threshold"], left=left_branch, right=right_branch)

        leaf_value = self.leaf_value_calculation(y)

        return DecisionNode(value=leaf_value)


   def predict_value(self, x, tree=None):
        if tree is None:
            tree = self.root
        # if leaf return value
        if tree.value is not None:
            return tree.value
        feature_value = x[tree.feature_i]

        branch = tree.right
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value <= tree.threshold:
                branch = tree.left
        elif feature_value == tree.threshold:
            branch = tree.left
        return self.predict_value(x, branch)

   def predict(self, X):
        y_pred = [self.predict_value(sample) for sample in X]
        return y_pred

   def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        # If we're at leaf => print the label
        if tree.value is not None:
            print (tree.value)
        else:
            # Print test
            print ("%s:%s? " % (tree.feature_i, tree.threshold))
            print ("%sT->" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print ("%sF->" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

class RegressionTree(DecisionTree):
    def _variance_reduction(self, y, y1, y2):
        var_tot = calculate_variance(y)
        var_1 = calculate_variance(y1)
        var_2 = calculate_variance(y2)
        frac_1 = len(y1) / len(y)
        frac_2 = len(y2) / len(y)

        # Calculate the variance reduction
        variance_reduction = var_tot - (frac_1 * var_1 + frac_2 * var_2)
        return sum(variance_reduction)

    def _mean_of_y(self, y):
        value = np.mean(y, axis=0)
        return value if len(value) > 1 else value[0]

    def fit(self, X, y):
        self.impurity_calculation = self._variance_reduction
        self.leaf_value_calculation = self._mean_of_y
        super(RegressionTree, self).fit(X, y)

class GradientBoostingForest(object): 
    def __init__(self, n_estimators=200, learning_rate=1, min_samples_split=2,
                 min_info_gain=1e-7,max_depth=2, debug=False,regression=True):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.min_samples_split = min_samples_split
        self.min_impurity = min_info_gain
        self.max_depth = max_depth
        self.regression = regression
        self.bar = progressbar.ProgressBar(widgets= [
                    'Training: ', progressbar.Percentage(), ' ', progressbar.Bar(marker="-", left="[", right="]"),
                    ' ', progressbar.ETA()])
        self.loss = SquareLoss()
        if not self.regression:
            self.loss = CrossEntropy()

        # Initialize regression trees
        self.trees = []
        for _ in range(n_estimators):
            tree = RegressionTree(
                    min_samples_split=self.min_samples_split,
                    min_impurity=self.min_impurity,
                    max_depth=self.max_depth)
            self.trees.append(tree)


    def fit(self, X, y):
        if self.regression==False:
          y = to_categorical(y)
          y_pred = np.full(np.shape(y), np.zeros((y.shape[0],1)))
        else:
          y = y.reshape(y.shape[0],1)
          y_pred = np.zeros((y.shape[0],1))
        for i in self.bar(range(self.n_estimators)):
          # get dL/dy_i_pred for EACH sample 
            gradient = self.loss.gradient(y, y_pred)
            self.trees[i].fit(X, gradient)
            update = self.trees[i].predict(X)
            if self.regression:
              update = np.array(update).reshape(len(update),1)
            #print(update.shape)
            # Update y prediction
            if i>0:
              y_pred -= np.multiply(self.learning_rate, update)
            else:
              y_pred -= update
  

    def predict(self, X):
        y_pred = np.array([])
        # Make predictions
        idx=0
        for tree in self.trees:
            update = tree.predict(X)
            if idx >0 :
              update = np.multiply(self.learning_rate, update)
            else:
              update=np.array(update)
            y_pred = -update if not y_pred.any() else y_pred - update
            idx = idx+1

        if not self.regression:
            y_pred = np.exp(y_pred) / np.expand_dims(np.sum(np.exp(y_pred), axis=1), axis=1)
            y_pred = np.argmax(y_pred, axis=1)
        return y_pred

class XGBoostTree(DecisionTree):

    def split(self, y):
        col = int(np.shape(y)[1]/2)
        y, y_pred = y[:, :col], y[:, col:]
        return y, y_pred

    def gain(self, y, y_pred):
        nominator = np.power((y * self.loss.gradient(y, y_pred)).sum(), 2)
        denominator = self.loss.hessian(y, y_pred).sum()
        return 0.5 * (nominator / (denominator+self.tree_lambda))

    def gain_by_taylor(self, y, y1, y2):
        # Split
        y, y_pred = self.split(y)
        y1, y1_pred = self.split(y1)
        y2, y2_pred = self.split(y2)

        true_gain = self.gain(y1, y1_pred)
        false_gain = self.gain(y2, y2_pred)
        gain = self.gain(y, y_pred)
        return 0.5*(true_gain + false_gain - gain)-self.tree_lambda

    def approximate_update(self, y):
        y, y_pred = self.split(y)
        gradient = np.sum(y * self.loss.gradient(y, y_pred), axis=0)
        hessian = np.sum(self.loss.hessian(y, y_pred), axis=0)
        update_approximation =  gradient / (hessian +self.tree_lambda)
        return update_approximation

    def fit(self, X, y):
        self.impurity_calculation = self.gain_by_taylor
        self.leaf_value_calculation = self.approximate_update
        super(XGBoostTree, self).fit(X, y)

class XGBoostForest(object):
    def __init__(self, n_estimators=200, learning_rate=0.001, min_samples_split=2,
                 min_impurity=1e-7, max_depth=2,tree_lambda=0.05,classify=True):
        self.n_estimators = n_estimators            # Number of trees
        self.learning_rate = learning_rate          # Step size for weight update
        self.min_samples_split = min_samples_split  # The minimum n of sampels to justify split
        self.min_impurity = min_impurity              # Minimum variance reduction to continue
        self.max_depth = max_depth                  # Maximum depth for tree

        self.bar = progressbar.ProgressBar(widgets= [
                    'Training: ', progressbar.Percentage(), ' ', progressbar.Bar(marker="-", left="[", right="]"),
                    ' ', progressbar.ETA()])
        self.loss = LogisticLoss()
        self.tree_lambda = tree_lambda
        self.classify = classify
        self.trees = []
        for i in range(n_estimators):
            tree = XGBoostTree(
                    classify =self.classify,
                    min_samples_split=self.min_samples_split,
                    min_impurity=min_impurity,
                    max_depth=self.max_depth,
                    loss=self.loss,
                    tree_lambda=self.tree_lambda)
            self.trees.append(tree)

    def fit(self, X, y):
        if self.classify:
          y = to_categorical(y)
        else:
          y = y.reshape(y.shape[0],1)
        y_pred = np.zeros(np.shape(y)) #np.full(np.shape(y), np.zeros((y.shape[0],1)))
        if self.classify:
            self.num_classes = y.shape[1]

        #y_pred = np.zeros(np.shape(y))
        for i in self.bar(range(self.n_estimators)):
            tree = self.trees[i]
            y_and_pred = np.concatenate((y, y_pred), axis=1)
            tree.fit(X, y_and_pred)
            update_pred = tree.predict(X)
            if i>0:
              y_pred -= np.multiply(self.learning_rate, update_pred)
            else:
               y_pred -= update_pred
            #print("y_pred: "+str(y_pred))

    def predict(self, X):
        y_pred = None
        idx=0
        for tree in self.trees:
            update = tree.predict(X)
            #update = np.array(update).reshape(np.shape(X)[0],self.num_classes)
            if y_pred is None:
                y_pred = np.zeros_like(update)
            if idx >0 :
              update = np.multiply(self.learning_rate, update)
            else:
              update=np.array(update)
            #print("update: "+str(update))
            y_pred = -update if not y_pred.any() else y_pred - update
            idx = idx+1
        #print(y_pred)
        if self.classify:
            y_pred = np.exp(y_pred) / np.sum(np.exp(y_pred), axis=1, keepdims=True)
            y_pred = np.argmax(y_pred, axis=1)
        return y_pred

In [ ]:
data = datasets.load_iris()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

clf = GradientBoostingForest(n_estimators=50,learning_rate=0.05,max_depth=2,regression=False)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred)
print(y_test)
accuracy = accuracy_score(y_test, y_pred)

print ("Accuracy:", accuracy)

###################################################
print("================Regression=================")
data = pd.read_csv('TempLinkoping2016.txt', sep="\t")

time = np.atleast_2d(data["time"].values).T
temp = np.atleast_2d(data["temp"].values).T

X = time.reshape((-1, 1))               # Time. Fraction of the year [0, 1]
X = np.insert(X, 0, values=1, axis=1)   # Insert bias term
y = temp[:, 0]                          # Temperature. Reduce to one-dim

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

model = GradientBoostingForest(n_estimators=50,learning_rate=0.05,max_depth=2,regression=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_pred_line = model.predict(X)
mse = mean_squared_error(y_test, y_pred)

print ("Mean Squared Error:", mse)

Training: 100% [-----------------------------------------------] Time:  0:00:03


[2 0 1 2 2 1 1 1 0 0 1 1 0 1 2 1 2 0 0 0 1 2 2 2 2 0 0 2 1 2 2 2 1 0 0 1 2
 1 0 1 1 1 0 1 2 1 1 2 2 2 1 2 1 2 1 2 1 0 1 0]
[2 0 1 2 2 1 1 1 0 0 1 1 0 1 2 1 2 0 0 0 1 2 2 2 2 0 0 2 1 2 2 2 2 0 0 1 2
 1 0 1 1 1 0 1 2 1 1 2 2 1 1 2 1 2 1 2 2 0 1 0]
Accuracy: 0.95


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:272: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:272: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
# data = datasets.load_iris()
# X = data.data
# y = data.target
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# clf = XGBoostForest(n_estimators=50,learning_rate=0.05,max_depth=2,tree_lambda=0.0001)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# print(y_pred)
# print(y_test)
# accuracy = accuracy_score(y_test, y_pred)

# print ("Accuracy:", accuracy)
###################################################
print("================Regression=================")
data = pd.read_csv('TempLinkoping2016.txt', sep="\t")

time = np.atleast_2d(data["time"].values).T
temp = np.atleast_2d(data["temp"].values).T

X = time.reshape((-1, 1))               # Time. Fraction of the year [0, 1]
X = np.insert(X, 0, values=1, axis=1)   # Insert bias term
y = temp[:, 0]                          # Temperature. Reduce to one-dim

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

model = XGBoostForest(n_estimators=50,learning_rate=0.01,max_depth=4,tree_lambda=0.0001,classify=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_pred_line = model.predict(X)
mse = mean_squared_error(y_test, y_pred)

print ("Mean Squared Error:", mse)

Training: N/A% [                                               ] ETA:  --:--:--

================Regression=================


Training: 100% [-----------------------------------------------] Time:  0:00:08


Mean Squared Error: 1.323393636282258e+16
